# Network

## 参考文献

* https://zhangbinalan.gitbooks.io/protocol/content/chapter1.html

## TCP

#### TCP 报文

![](https://bj.bcebos.com/ipic/TCP报文格式.gif)

### TCP 有限状态机

<img src="https://bj.bcebos.com/ipic/TCP状态机.jpeg" width="70%">

#### CLOSE_WAIT

**CLOSE_WAIT是TCP被动关闭的一方在收到FIN包回复ACK后进入的状态，如果服务处于该状态的连接过多，一般可能是应用程序没有或不能正常关闭TCP连接，需要重点关注。**

#### TIME_WAIT

TIME_WAIT是主动关闭TCP的一方发送回复对方FIN发送ACK之后进入的状态。需要等待2MSL释放该连接。

什么是2MSL？

MSL（Maximum Segment Lifetime）最大报文生存时间。主动关闭的一方需要等待2MSL才能从TIME_WAIT状态变迁到CLOSED状态。

为什么需要2MSL？

我们假设网络是不可靠的，因此无法保证最后的ACK报文一定能被对方收到，当对方处于LAST_ACK状态的socket可能因为超时重新发送FIN报文，当socket处于TIME_WAIT状态时，可以接受对方的FIN重新发送ACK。

## HTTP

*https://developer.mozilla.org/zh-CN/docs/Web/HTTP/Overview*

![](https://media.prod.mdn.mozit.cloud/attachments/2016/08/09/13687/5d4c4719f4099d5342a5093bdf4a8843/HTTP_Request.png)
    

![](https://media.prod.mdn.mozit.cloud/attachments/2016/08/09/13691/58390536967466a1a59ba98d06f43433/HTTP_Response.png)

### HTTPS

*https://zhuanlan.zhihu.com/p/102597710*

HTTPS分为两个大的步骤：

1. 非对称的加解密
2. 对称加解密

![](https://pic2.zhimg.com/80/v2-cd198e0929f583fb87cffe9686edf901_hd.jpg)

## NetFilter

Netfilter是内核防火墙框架，可以实现网络安全策略的很多功能，包括包过滤、数据包处理、地址伪装、逃命代理、网络地址转换（NAT）等。

iptables是基于Netfilter接口配置Netfilter配置表。

iptables内置了4张表和5条链，4张表分别为RAW、Mangle、NAT和Filter表，5条链又叫做数据包的5个挂载点（Hook Point，可以将其理解为回调函数点，再数据包到达这些位置时，内核会主动调用回调函数），分别是PREROUTING、INPUT、OUTPUT、FORWARD和POSTROUTING。

对于不同表的相同链执行顺序，iptables定义了优先级，该优先级从高到低分别raw、mangle、nat和filter。例如，对于PREROUTING链，首先执行raw表的规则，然后执行mangle表的规则，最后执行nat表的规则。

![](https://bj.bcebos.com/ipic/iptables表和链的关系.png)

1. Filter表

Filter表是iptables的默认表，如果在创建规则时未指定表，那么默认使用Filter表，主要用于包过滤、根据具体规则决定是否放行该数据包（DROP、ACCEPT、REJECT、LOG）

Filter表包含了如下三种内建链：

* INPUT链：过滤目的地址是本地的所有数据包
* OUTPUT链：过滤本机产生的所有数据包
* FORWARD链：过滤经过本机的所有数据包（原地址和目的地址都不是本机）

2. NAT表

NAT表主要用于修改数据包的IP地址、端口的呢更信息，包含了以下三种内建链：

* PREROUTING链：DNAT，处理刚到本机并在路由转发前转换数据包的目的地址
* POSTORUTING链：SNAT，处理即将离开本机的数据包，转换数据包的原地址
* OUTPUT链：MASQUERADE，改变本地产生的数据包的原地址

3. Mangle表

主要用于修改数据包的TOS、TTL，以及数据包设置Mark标记，已实现QoS调整以及策略路由，它包含了PREROUTING、POSTOURTING、INPUT、OUTPUT和FORWARD。

4. Raw表

Raw表是iptables在1.2.9版本后新增的表，主要用于决定数据包是否被状态跟踪机制处理。RAW表的规则要优于其他表，包含了两条规则链：OUTPUT和PREROUTING。

iptables规则链处理数据包的时机：

![iptables规则链处理数据包的时机](https://bj.bcebos.com/ipic/iptables规则链处理数据包的时机.jpg)